In [43]:
# %matplotlib notebook

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import cv2
import csv
from matplotlib import pyplot as plt

## Import previous data

In [44]:
def importRecords():
    a = csv.DictReader(open('data.csv', encoding='utf-8'), delimiter=',', skipinitialspace=True)
    return [x for x in a]

## Website management

In [45]:
def getCountry(name):
    for r in records:
        if r["country"] == name:
#             print(r)
            return [r["Xcentroid"], r["Ycentroid"]]
        
    return False

def saveCountry(name, Xcentroid, Ycentroid):
    records.append({'country': name, 'Xcentroid': Xcentroid, 'Ycentroid':Ycentroid})
    
def saveRecords():
    with open('data.csv', 'w', newline='', encoding='utf-8') as f:
        dict_writer = csv.DictWriter(f, records[0].keys())
        dict_writer.writeheader()
        dict_writer.writerows(records)
        
#print(getCountry("Quentin"))
#print(getCountry("Unkown"))

#saveCountry("Emile", 12, 15)
#print(getCountry("Emile"))
#saveRecords()

In [46]:
def initGame(game):
    # Initialize the webdriver
    options = Options()
#     options.add_argument(r"--user-data-dir=C:\Users\emile\AppData\Local\Google\Chrome\User Data\Default")
    options.add_extension(r'.\extensions\1.42.4_1.crx')
    options.add_argument("--start-maximized")
    options.add_experimental_option("prefs", { 
        "profile.default_content_setting_values.notifications": 1 
    })
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)

    # Open the website
    
    browser.get('https://www.jeux-geographiques.com/jeux-en-ligne-Jeu-Pays-d-Afrique-_pageid158.html')
#     browser.get('https://www.jeux-geographiques.com/jeux-en-ligne-Jeu-Pays-d-Amerique-du-Sud-_pageid162.html')
    cookies_button = browser.find_element(By.XPATH, "//button[text()=\"J'ACCEPTE\"]").click()
    print("login time")
    time.sleep(25)
    print("login time over")
    browser.execute_script("document.getElementById('canvas').scrollIntoView();")
    
    if game == 'Win' :
        browser.find_element(By.ID,'buttonStart').click()
    else : 
        browser.find_element(By.ID,'buttonStartWithoutTimer').click()
    
    return browser

def closeGame(browser):
    browser.quit()

## Game management

In [47]:
w_page = 1920
h_page = 1080

y_scroll = 1076

x_canva = 310
y_canva = 1076
w_canva = 900
h_canva = 500

x_crop = 0
y_crop = 50
w_crop = w_canva
h_crop = h_canva

def screenshotToSeleniumCoordinates(x_screenshot,y_screenshot) :
    # transposes screenshot xy coordinates, to selenium xy coordinates (relative to center of element(canvas))
    x_selenium = x_screenshot-448
    y_selenium = y_screenshot-248
    return x_selenium,y_selenium

def findCountryPositionOld():
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.4)
    canvas.screenshot("screenshot.PNG")

    background = cv2.imread('bg_africa.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]
#     background = cv2.imread('bg_southamerica.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]
    after = cv2.imread('screenshot.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]

    diff = cv2.absdiff(background, after)
    ret,thresh = cv2.threshold(diff,10,255,cv2.THRESH_BINARY)
    canny_edges = cv2.Canny(thresh, 30, 200)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    cXlist=[]
    cYlist=[]

    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cXlist.append(cX)
            cYlist.append(cY)
        else:
            pass # do nothing, centroid value is not usable
            
    if len(cXlist) > 0:
#         cv2.circle(diff, (cXlist[0], cYlist[0]), 5, (255, 0, 0), -1)
#         cv2.putText(diff, "centroid", (cXlist[0] - 25, cYlist[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#         plt.imshow(diff)
#         plt.show()
        return cXlist[0],cYlist[0]
    else:
        return False # no centroid found in list 
    
def sort_contours(contours, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM'):
    # initialize the reverse flag
    x_reverse = False
    y_reverse = False
    if x_axis_sort == 'RIGHT_TO_LEFT':
        x_reverse = True
    if y_axis_sort == 'BOTTOM_TO_TOP':
        y_reverse = True
    
    boundingBoxes = [cv2.boundingRect(c) for c in contours]
    
    # sorting on x-axis 
    sortedByX = zip(*sorted(zip(contours, boundingBoxes),
    key=lambda b:b[1][0], reverse=x_reverse))
    
    # sorting on y-axis 
    (contours, boundingBoxes) = zip(*sorted(zip(*sortedByX),
    key=lambda b:b[1][1], reverse=y_reverse))
    # return the list of sorted contours and bounding boxes
    return (contours, boundingBoxes)

    
def clickOnCanva(x_click, y_click):
    action.move_by_offset(x_click,y_click).click().perform()
    action.move_by_offset(-int(x_click),-int(y_click))

def findCountryPosition():
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.1)
    action.move_to_element_with_offset(canvas,0,0).click().perform()
    time.sleep(0.4)
    canvas.screenshot("screenshot.PNG")

    background = cv2.imread('bg_africa.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]
    after = cv2.imread('screenshot.PNG',0)[y_crop:y_crop+h_crop, x_crop:x_crop+w_crop]
    
    diff = cv2.absdiff(background, after)
    ret,thresh = cv2.threshold(diff,10,255,cv2.THRESH_BINARY)
    import numpy as np
    kernel = np.ones((4,4), np.uint8) # Creating kernel
    eroded = cv2.erode(thresh, kernel)
    
    contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(list(contours)) > 0 :
        contours, boundingBoxes = sort_contours(contours, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='BOTTOM_TO_TOP')
    else :
        pass # no contours found

    cXlist=[]
    cYlist=[]

    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cXlist.append(cX)
            cYlist.append(cY)
        else:
            pass # do nothing, centroid value is not usable

    if len(cXlist) > 0:
#         cv2.circle(diff, (cXlist[0], cYlist[0]), 5, (255, 0, 0), -1)
#         cv2.putText(diff, "centroid", (cXlist[0] - 25, cYlist[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#         plt.imshow(diff)
#         plt.show()
        return cXlist[0],cYlist[0]
    else:
        return False # no centroid found in list 
    
def actionCountry(name):
    print(name)
    country_coordinates = getCountry(name)
    
    if not country_coordinates:
        print("Country not found in records, trying to find it ...")
        coordinates = findCountryPosition()
        
        if not coordinates:
            print("Unable to find the centroid")
            
        else:
            
            x_click,y_click = screenshotToSeleniumCoordinates(coordinates[0],coordinates[1])
            print("Centroid found, adding country to records :")
            print(name, coordinates)
#             print(coordinates[0],coordinates[1])
            saveCountry(name, x_click, y_click)
        
    else:
        clickOnCanva(country_coordinates[0], country_coordinates[1])
        print("Country found in records :")
        print(name, country_coordinates)
        
    print("\n")



## Run

### Learn then win

In [48]:
records = importRecords()

## Learn & win
# for game in ["Learn" , "Win"]:
for game in ["Win"]:
    print(game)
    browser = initGame(game)
    time.sleep(1.5)
    action = ActionChains(browser)
    canvas = browser.find_element(By.ID,'canvas')
    if game == 'Learn' :
        canvas.screenshot("bg_africa.PNG") # save background for later
    action.move_to_element(canvas).perform()
    
    for i in range(57):
        if i != 0 :
            time.sleep(1.65) # wait for country name to appear
        name = browser.find_element(By.ID,'questionTextLabel').text
        actionCountry(name)

    saveRecords()
    
#     time.sleep(2)
#     closeGame(browser)
    



====== WebDriver manager ======


Win


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\emile\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\emile\AppData\Local\Temp\ipykernel_4464\2881642056.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)


login time
login time over
Sao Tomé et Principe
Country found in records :
Sao Tomé et Principe ['-88', '15']


Le Maroc
Country found in records :
Le Maroc ['-156', '-208']


Le Botswana
Country found in records :
Le Botswana ['27', '159']


Le Mali
Country found in records :
Le Mali ['-152', '-103']


La Libye
Country found in records :
La Libye ['-10', '-165']


Le Congo-Kinshasa
Country found in records :
Le Congo-Kinshasa ['0', '0']


La Namibie
Country found in records :
La Namibie ['-13', '166']


Le Congo-Brazzaville
Country found in records :
Le Congo-Brazzaville ['-37', '9']


La Gambie
Country found in records :
La Gambie ['-227', '-79']


Le Nigéria
Country found in records :
Le Nigéria ['-79', '-50']


Le Sahara occidental
Country found in records :
Le Sahara occidental ['-184', '-168']


L'Algérie
Country found in records :
L'Algérie ['-105', '-172']


La Rép. Centrafricaine
Country found in records :
La Rép. Centrafricaine ['13', '-36']


L'Ouganda
Country found in recor